### 平面输入

In [1]:
from langchain.chains import LLMChain
from langchain.llms import ChatGLM
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [2]:
template = """您是一个正在与人类对话的聊天机器人.

{chat_history}
人类: {human_input}
聊天机器人:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

> **memory_key, 指明了,要从memory中取的key**

In [3]:
prompt.input_variables

['chat_history', 'human_input']

In [4]:
PromptTemplate.from_template(template).input_variables

['chat_history', 'human_input']

> 注意这里用的是chat_history 默认是 history

In [3]:
ConversationBufferMemory?

Init signature:
ConversationBufferMemory(
    *,
    chat_memory: langchain.schema.chat_history.BaseChatMessageHistory = None,
    output_key: Optional[str] = None,
    input_key: Optional[str] = None,
    return_messages: bool = False,
    human_prefix: str = 'Human',
    ai_prefix: str = 'AI',
    memory_key: str = 'history',
) -> None
Docstring:      Buffer for storing conversation memory.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/anaconda3/envs/lglm/lib/python3.9/site-packages/langchain/memory/buffer.py
Type:           ModelMetaclass
Subclasses:     ZepMemory

In [4]:
llm = ChatGLM()
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [5]:
llm_chain({'human_input': "你好,我的朋友"})



> Entering new LLMChain chain...
Prompt after formatting:
您是一个正在与人类对话的聊天机器人.


人类: 你好,我的朋友
聊天机器人:

> Finished chain.


{'human_input': '你好,我的朋友',
 'chat_history': '',
 'text': '你好！很高兴能与您交流，请问有什么我可以帮助您的吗？'}

> **第一次调用chat_history是为空的**

In [7]:
llm_chain({'human_input': "还不错, 你好么"})



> Entering new LLMChain chain...
Prompt after formatting:
您是一个正在与人类对话的聊天机器人.

Human: 你好,我的朋友
AI: 你好！很高兴能与您交流，请问有什么我可以帮助您的吗？
人类: 还不错, 你好么
聊天机器人:

> Finished chain.


{'human_input': '还不错, 你好么',
 'chat_history': 'Human: 你好,我的朋友\nAI: 你好！很高兴能与您交流，请问有什么我可以帮助您的吗？',
 'text': '作为聊天机器人,我很高兴能够与您交流,并尽力回答您的问题或提供帮助。请问有什么我可以帮忙的吗?'}

> **第二论对话, 会将第一轮对话作为历史保存(其实是在第一轮对话的末尾,被保存到memory中的)**

In [8]:
memory.memory_variables

['chat_history']

### 结构化的输入

In [9]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

In [10]:
from langchain.llms import ChatGLM

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="您是一个正在与人类对话的聊天机器人."), #  The persistent system prompt
        MessagesPlaceholder(variable_name='chat_history'),         # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{human_input}"), # # Where the human input will injected
    ]
)

In [12]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [13]:
llm = ChatGLM()

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [15]:
chat_llm_chain({'human_input': "你好,我的朋友"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一个正在与人类对话的聊天机器人.
Human: 你好,我的朋友

> Finished chain.


{'human_input': '你好,我的朋友',
 'chat_history': [HumanMessage(content='你好,我的朋友'),
  AIMessage(content='你好,朋友!请问有什么需要帮助的吗?')],
 'text': '你好,朋友!请问有什么需要帮助的吗?'}

In [16]:
chat_llm_chain({'human_input': "还不错, 你好么"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一个正在与人类对话的聊天机器人.
Human: 你好,我的朋友
AI: 你好,朋友!请问有什么需要帮助的吗?
Human: 还不错, 你好么

> Finished chain.


{'human_input': '还不错, 你好么',
 'chat_history': [HumanMessage(content='你好,我的朋友'),
  AIMessage(content='你好,朋友!请问有什么需要帮助的吗?'),
  HumanMessage(content='还不错, 你好么'),
  AIMessage(content='AI: 作为聊天机器人,我并没有情感,所以不能说我“好”或者“不好”。不过,我很乐意与您聊天,并尽力回答您的问题。请问有什么需要帮助的吗?')],
 'text': 'AI: 作为聊天机器人,我并没有情感,所以不能说我“好”或者“不好”。不过,我很乐意与您聊天,并尽力回答您的问题。请问有什么需要帮助的吗?'}

### Memory in the Multi-Input Chain: context and human input

In [1]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [2]:
path = "/home/10170464/jupyter-notebook/Wangquanjun/langchain-ChatGLM/huggingface_tools/dataroot/models/GanymedeNil/text2vec-large-chinese"
embeddings = HuggingFaceEmbeddings(model_name=path,
                                   model_kwargs={'device': "cpu"})

No sentence-transformers model found with name /home/10170464/jupyter-notebook/Wangquanjun/langchain-ChatGLM/huggingface_tools/dataroot/models/GanymedeNil/text2vec-large-chinese. Creating a new one with MEAN pooling.


In [3]:
with open("./inputs/state_of_chinese.txt") as f:
    state_of_chinese = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_chinese)

In [4]:
docsearch = FAISS.from_texts(texts, embeddings, metadatas=[{"source": i} for i in range(len(texts))])

In [5]:
query = "AIGC能替代老师么"
docs = docsearch.similarity_search(query)

In [6]:
docs

[Document(page_content='“教育是一个国家的未来，是一个民族的希望。”我相信，每一位阅读这篇文章的人，无论是在校学生，家长，还是教育工作者，都深知教育的重要性。\n\n然而，传统的教育模式面临着许多挑战：课堂内的教学模式单一，教师难以顾及每一位学生的个性化需求，学生的学习进度和成绩又受到诸多因素的影响。而这些问题，可能通过AIGC技术得到解决。AIGC，全称为人工智能生成内容，是现代AI技术的重要应用之一。\n\n那么，AIGC技术在教育领域有哪些应用？AIGC技术在教育领域的优缺点又是什么呢？在教育领域，AI是否能替代人类教师的角色？？AIGC教育领域的应用可能带来哪些伦理问题呢？这些问题，我将在接下来的文章中为大家一一解答。\n\n我们首先来看看AIGC技术在教育领域的应用。Quizlet联合OpenAI推出了名为Q-Chat的人工智能辅导工具，这个工具可以提供个性化的一对一辅导，通过聊天界面和针对性的问题，帮助学生学习各个学科。这不仅仅是一种新的学习方式，更是一种全新的学习体验。Q-Chat的出现，让我们看到了AI在教育领域的巨大潜力。\n\n同样，在语言学习领域，Duolingo也引入了OpenAI的GPT-4，开发了AI对话伙伴和解释答案两个新功能，以提升学习者在学习第二语言时的效率和兴趣。另外，非营利组织Khan Academy宣布将使用GPT-4推出AI助手Khanmigo，作为学生的虚拟导师和教师的课堂助手。\n\n那么，AIGC技术在教育领域应用有哪些优缺点呢？\n\n优点显而易见，首先，AIGC技术可以实现个性化学习，提供定制化的教学内容，提升学习效率。其次，AI助手可以提供全天候服务，打破时间和地点的限制，让学习更为灵活，学生可以随时随地进行学习。\n\n然而，任何事物的发展都有其两面性，缺点同样存在。例如，过度依赖AI可能会导致学生缺乏人际交流的机会，影响他们的社交能力。这不仅包括语言交流，还有面对面的情感交流、情绪理解和社会性行为。人类社会是一个复杂的系统，充满了微妙的情感和社会关系，这是AI暂时无法完全理解和模拟的。\n\n面对这种情况，我们要思考，AIGC技术是否能替代人类教师的角色？', metadata={'source': 0}),
 Document(page_content='我认为，AIGC技术并不能完

In [7]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import ChatGLM
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [8]:
template = """您是一个正在与人类对话的聊天机器人.

在给定的下文中，创建最终答案。

{context}

{chat_history}
人类: {human_input}
聊天机器人:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

In [10]:
chain = load_qa_chain(
    ChatGLM(), chain_type="stuff", memory=memory, prompt=prompt
)

> <font color=blue>这是最终丢给LLM的prompt</font>

In [24]:
chain.llm_chain.prompt

PromptTemplate(input_variables=['chat_history', 'context', 'human_input'], template='您是一个正在与人类对话的聊天机器人.\n\n在给定的下文中，创建最终答案。\n\n{context}\n\n{chat_history}\n人类: {human_input}\n聊天机器人:')

In [ ]:
>  那什么时候组装的呢?

In [11]:
chain({"input_documents": docs, "human_input": query})

{'input_documents': [Document(page_content='“教育是一个国家的未来，是一个民族的希望。”我相信，每一位阅读这篇文章的人，无论是在校学生，家长，还是教育工作者，都深知教育的重要性。\n\n然而，传统的教育模式面临着许多挑战：课堂内的教学模式单一，教师难以顾及每一位学生的个性化需求，学生的学习进度和成绩又受到诸多因素的影响。而这些问题，可能通过AIGC技术得到解决。AIGC，全称为人工智能生成内容，是现代AI技术的重要应用之一。\n\n那么，AIGC技术在教育领域有哪些应用？AIGC技术在教育领域的优缺点又是什么呢？在教育领域，AI是否能替代人类教师的角色？？AIGC教育领域的应用可能带来哪些伦理问题呢？这些问题，我将在接下来的文章中为大家一一解答。\n\n我们首先来看看AIGC技术在教育领域的应用。Quizlet联合OpenAI推出了名为Q-Chat的人工智能辅导工具，这个工具可以提供个性化的一对一辅导，通过聊天界面和针对性的问题，帮助学生学习各个学科。这不仅仅是一种新的学习方式，更是一种全新的学习体验。Q-Chat的出现，让我们看到了AI在教育领域的巨大潜力。\n\n同样，在语言学习领域，Duolingo也引入了OpenAI的GPT-4，开发了AI对话伙伴和解释答案两个新功能，以提升学习者在学习第二语言时的效率和兴趣。另外，非营利组织Khan Academy宣布将使用GPT-4推出AI助手Khanmigo，作为学生的虚拟导师和教师的课堂助手。\n\n那么，AIGC技术在教育领域应用有哪些优缺点呢？\n\n优点显而易见，首先，AIGC技术可以实现个性化学习，提供定制化的教学内容，提升学习效率。其次，AI助手可以提供全天候服务，打破时间和地点的限制，让学习更为灵活，学生可以随时随地进行学习。\n\n然而，任何事物的发展都有其两面性，缺点同样存在。例如，过度依赖AI可能会导致学生缺乏人际交流的机会，影响他们的社交能力。这不仅包括语言交流，还有面对面的情感交流、情绪理解和社会性行为。人类社会是一个复杂的系统，充满了微妙的情感和社会关系，这是AI暂时无法完全理解和模拟的。\n\n面对这种情况，我们要思考，AIGC技术是否能替代人类教师的角色？', metadata={'source': 0}),
  Document(page_co

In [12]:
chain.memory.buffer

'Human: AIGC能替代老师么\nAI: AIGC技术在教育领域的应用可以提供个性化教学，但无法完全替代人类教师的角色。'

In [13]:
chain({"input_documents": docs, "human_input": "老师的优势表现在哪些地方"})

{'input_documents': [Document(page_content='“教育是一个国家的未来，是一个民族的希望。”我相信，每一位阅读这篇文章的人，无论是在校学生，家长，还是教育工作者，都深知教育的重要性。\n\n然而，传统的教育模式面临着许多挑战：课堂内的教学模式单一，教师难以顾及每一位学生的个性化需求，学生的学习进度和成绩又受到诸多因素的影响。而这些问题，可能通过AIGC技术得到解决。AIGC，全称为人工智能生成内容，是现代AI技术的重要应用之一。\n\n那么，AIGC技术在教育领域有哪些应用？AIGC技术在教育领域的优缺点又是什么呢？在教育领域，AI是否能替代人类教师的角色？？AIGC教育领域的应用可能带来哪些伦理问题呢？这些问题，我将在接下来的文章中为大家一一解答。\n\n我们首先来看看AIGC技术在教育领域的应用。Quizlet联合OpenAI推出了名为Q-Chat的人工智能辅导工具，这个工具可以提供个性化的一对一辅导，通过聊天界面和针对性的问题，帮助学生学习各个学科。这不仅仅是一种新的学习方式，更是一种全新的学习体验。Q-Chat的出现，让我们看到了AI在教育领域的巨大潜力。\n\n同样，在语言学习领域，Duolingo也引入了OpenAI的GPT-4，开发了AI对话伙伴和解释答案两个新功能，以提升学习者在学习第二语言时的效率和兴趣。另外，非营利组织Khan Academy宣布将使用GPT-4推出AI助手Khanmigo，作为学生的虚拟导师和教师的课堂助手。\n\n那么，AIGC技术在教育领域应用有哪些优缺点呢？\n\n优点显而易见，首先，AIGC技术可以实现个性化学习，提供定制化的教学内容，提升学习效率。其次，AI助手可以提供全天候服务，打破时间和地点的限制，让学习更为灵活，学生可以随时随地进行学习。\n\n然而，任何事物的发展都有其两面性，缺点同样存在。例如，过度依赖AI可能会导致学生缺乏人际交流的机会，影响他们的社交能力。这不仅包括语言交流，还有面对面的情感交流、情绪理解和社会性行为。人类社会是一个复杂的系统，充满了微妙的情感和社会关系，这是AI暂时无法完全理解和模拟的。\n\n面对这种情况，我们要思考，AIGC技术是否能替代人类教师的角色？', metadata={'source': 0}),
  Document(page_co

In [15]:
print(chain.memory.buffer)

Human: AIGC能替代老师么
AI: AIGC技术在教育领域的应用可以提供个性化教学，但无法完全替代人类教师的角色。
Human: 老师的优势表现在哪些地方
AI: 优点包括：
1. 个性化教学：AIGC技术可以提供定制化的教学内容，帮助学生学习各个学科。
2. 提高学习效率：AIGC技术可以实现个性化学习，提供针对性的教学内容，提高学习效率。
3. 打破时间和地点的限制：AIGC技术可以提供全天候服务，打破时间和地点的限制，让学习更为灵活。
4. 减轻教师负担：AIGC技术可以提供一些辅助教学功能，如编写课堂提示、创建教学材料等，减轻教师负担。

缺点包括：
1. 缺乏人际交流的机会：AIGC技术可以提供聊天界面，但无法提供面对面的交流机会，可能会影响学生的社交能力。
2. 社交能力不足：AIGC技术无法提供面对面的情感交流和社会性行为，可能会影响学生的社交能力。
3. 无法完全理解和模拟情感和社会关系：AIGC技术无法完全理解和模拟情感和社会关系，无法代替人类教师的角色。

AIGC技术在教育领域的应用可以提供一些优势，如个性化教学、提高学习效率等，但无法完全替代人类教师的角色。教师在教育领域具有独特的优势，如情感交流、价值引导和兴趣激发等。因此，AIGC技术和人类教师应该相辅相成，形成更全面的教育体系。


In [16]:
chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='AIGC能替代老师么'), AIMessage(content='AIGC技术在教育领域的应用可以提供个性化教学，但无法完全替代人类教师的角色。'), HumanMessage(content='老师的优势表现在哪些地方'), AIMessage(content='优点包括：\n1. 个性化教学：AIGC技术可以提供定制化的教学内容，帮助学生学习各个学科。\n2. 提高学习效率：AIGC技术可以实现个性化学习，提供针对性的教学内容，提高学习效率。\n3. 打破时间和地点的限制：AIGC技术可以提供全天候服务，打破时间和地点的限制，让学习更为灵活。\n4. 减轻教师负担：AIGC技术可以提供一些辅助教学功能，如编写课堂提示、创建教学材料等，减轻教师负担。\n\n缺点包括：\n1. 缺乏人际交流的机会：AIGC技术可以提供聊天界面，但无法提供面对面的交流机会，可能会影响学生的社交能力。\n2. 社交能力不足：AIGC技术无法提供面对面的情感交流和社会性行为，可能会影响学生的社交能力。\n3. 无法完全理解和模拟情感和社会关系：AIGC技术无法完全理解和模拟情感和社会关系，无法代替人类教师的角色。\n\nAIGC技术在教育领域的应用可以提供一些优势，如个性化教学、提高学习效率等，但无法完全替代人类教师的角色。教师在教育领域具有独特的优势，如情感交流、价值引导和兴趣激发等。因此，AIGC技术和人类教师应该相辅相成，形成更全面的教育体系。')]), input_key='human_input', memory_key='chat_history')

In [19]:
print(chain.memory.buffer_as_messages)

[HumanMessage(content='AIGC能替代老师么'), AIMessage(content='AIGC技术在教育领域的应用可以提供个性化教学，但无法完全替代人类教师的角色。'), HumanMessage(content='老师的优势表现在哪些地方'), AIMessage(content='优点包括：\n1. 个性化教学：AIGC技术可以提供定制化的教学内容，帮助学生学习各个学科。\n2. 提高学习效率：AIGC技术可以实现个性化学习，提供针对性的教学内容，提高学习效率。\n3. 打破时间和地点的限制：AIGC技术可以提供全天候服务，打破时间和地点的限制，让学习更为灵活。\n4. 减轻教师负担：AIGC技术可以提供一些辅助教学功能，如编写课堂提示、创建教学材料等，减轻教师负担。\n\n缺点包括：\n1. 缺乏人际交流的机会：AIGC技术可以提供聊天界面，但无法提供面对面的交流机会，可能会影响学生的社交能力。\n2. 社交能力不足：AIGC技术无法提供面对面的情感交流和社会性行为，可能会影响学生的社交能力。\n3. 无法完全理解和模拟情感和社会关系：AIGC技术无法完全理解和模拟情感和社会关系，无法代替人类教师的角色。\n\nAIGC技术在教育领域的应用可以提供一些优势，如个性化教学、提高学习效率等，但无法完全替代人类教师的角色。教师在教育领域具有独特的优势，如情感交流、价值引导和兴趣激发等。因此，AIGC技术和人类教师应该相辅相成，形成更全面的教育体系。')]
